In [1]:
import json
import ast
import numpy as np
import pandas as pd
import nltk
import spacy
from spacy import displacy
from collections import Counter
import random;random.seed(1)
import en_core_web_sm
import textacy
import re
import unidecode
from text2digits import text2digits
t2d = text2digits.Text2Digits()
nlp = en_core_web_sm.load()


In [2]:
#with open('squad-dev-v1.1.json', 'r') as f:
    #df = json.load(f)
with open('wiki_00-qa-409.json', 'r') as f:
    df = json.load(f)

In [3]:
#build dataframe for squad to get random sample of 100
topics = []
paragraphs = []
questions = []
qids = []
answers = []
autoq = pd.DataFrame()
for title in range(len(df['data'])):
    for p in range(len(df['data'][title]['paragraphs'])):
        if df['data'][title]['paragraphs'][p]['qas'] == []:
            topics.append(df['data'][title]['title'])
            paragraphs.append(df['data'][title]['paragraphs'][p]['context'])
            questions.append('')
            answers.append('')
            qids.append('')
        else:
            for qa in range(len(df['data'][title]['paragraphs'][p]['qas'])):
                topics.append(df['data'][title]['title'])
                paragraphs.append(df['data'][title]['paragraphs'][p]['context'])
                questions.append(df['data'][title]['paragraphs'][p]['qas'][qa]['question'])
                qids.append(df['data'][title]['paragraphs'][p]['qas'][qa]['id'])
                answers.append(df['data'][title]['paragraphs'][p]['qas'][qa]['answers'][0]['text'])
autoq['topics'] = topics
autoq['paragraphs'] = paragraphs
autoq['questions'] = questions
autoq['id'] = qids
autoq['answers'] = answers
autoq.head()
#sample100 = squad.sample(100, random_state = 5)
#new_index = [pd.Index(list(range(100)))]
#sample100.set_index(new_index, inplace=True)
#sample100.head()

,topics,paragraphs,questions,id,answers
0,Anarchism,Anarchism is a political philosophy that advoc...,,,
1,Anarchism,"While opposition to the state is central, many...",,,
2,Anarchism,Anarchism does not offer a fixed body of doctr...,,,
3,Anarchism,"The word ""anarchism"" is composed from the word...",,,
4,Anarchism,The first political philosopher to call himsel...,,,


In [4]:
def preprocess(answer):
    prep_answer = answer.lstrip('\'')
    prep_answer = prep_answer.rstrip('.')
    prep_answer = prep_answer.replace('\""','')
    prep_answer = prep_answer.replace('\"','')
    prep_answer = prep_answer.replace('\"','')
    prep_answer = prep_answer.lower()
    return prep_answer

In [38]:
def check_answer_type(raw_answer):
    answer_type = []
#     prep_answer = unidecode.unidecode(preprocess(raw_answer))
    tokenized_answer = nlp(raw_answer)
    pos_answer = [token.pos_ for token in tokenized_answer]
    month_list = ['january', 'jan', 'february', 'feb', 'march', 'april', \
                                   'may','june', 'july', 'august', 'aug', 'september', 'sept',\
                                   'october', 'oct', 'nov', 'dec','november', 'december']
    if (pos_answer[0]=='VERB') and (len(raw_answer)<2):
        answer_type.append('VERB')
    elif (pos_answer[0]=='VERB') and (len(raw_answer)>=2):
        answer_type.append('VP')
    # conditional for "to be" verbs
    elif (pos_answer[0]=='PART') and (pos_answer[1]=='VERB'):
        answer_type.append('VP')
    elif (pos_answer[0]=='ADV'):
        if len(tokenized_answer)>1:
            if (pos_answer[1]=='ADJ'):
                answer_type.append('ADJ')
            elif (pos_answer[1]=='VERB'):
                answer_type.append('VP')
            else:
                answer_type.append('ADV')
        else:
            answer_type.append('ADV')
    elif pos_answer[0]=='PROPN':
        # Check if it's a date
        if str(tokenized_answer[0]).lower() in month_list:
            answer_type.append('DATE')
        else:
            answer_type.append('PROPN')
    elif (pos_answer[0]=='NUM') |  (pos_answer[0]=='PUNCT'):
        month_present = [1 for i in month_list if i in raw_answer.lower()]
        time_present = [1 for i in ['minutes', 'hours', 'seconds'] if i in raw_answer]
        if month_present!=[]:
            answer_type.append('DATE')
        elif time_present!=[]:
            answer_type.append('TIME')
        else:
            answer_type.append('NUM')
    elif pos_answer[0]=='ADJ':
        answer_type.append('ADJ')
    elif (pos_answer[0]=='NOUN') | (pos_answer[0]=='X'):
        # check if it's a hyphen-separated adjective
        if re.findall(r'\w+(?:-\w+)+'.lower(),raw_answer):
            answer_type.append('ADJ')
        # check if there's a digit in the answer
        elif bool(re.search(r'\d', raw_answer)):
             # check for currency symbols
            if bool(re.search(r'([£\$€])', raw_answer)):
                answer_type.append('MONEY')
            else:
                answer_type.append('YEAR')
        else:
            answer_type.append('NOUN')
    elif pos_answer[0]=='ADP':
        answer_type.append('ADP')
    elif pos_answer[0]=='DET':
        answer_type.append('NOUN')
    elif pos_answer[0]=='SYM':
        answer_type.append('MONEY')
    else:
        answer_type.append('Unknown type')
    return answer_type
    

In [39]:
def generate_distractor(topic, paragraph, question, answer, answer_type):
    wrong_answers=[]
    #preprocess answer
    correct_answer = nlp(preprocess(answer))
    # get answer pos
    ans_length = len(correct_answer)
    ans_tag = [token.tag_ for token in correct_answer]
    ans_pos = [token.pos_ for token in correct_answer]
    # tokenize paragraph
    article = nlp(unidecode.unidecode(paragraph))
    doc = textacy.Doc(paragraph, lang='en_core_web_sm')
        
    # Preprocessing for same sentence distractor generation
    # get all sentences in paragraph
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_list = sent_detector.tokenize(paragraph.strip())

    # find sentence that has answer
    for s in sent_list:
        if answer in s:
            ans_sent = s
    # tokenize sentence
    sentence = nlp(ans_sent)
    sent = textacy.Doc(sentence, lang='en_core_web_sm')
        
    # Preprocessing for same topic distractor generation
    # choose random paragraph from same topic
    topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == topic), None)
    index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
    # make sure isn't the same paragraph as current paragraph
    p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                    if d["context"] == paragraph), None)
    index_list.remove(p_index)
    # Choose 5 random paragraphs from the same article
    alt_p_index = random.choices(index_list,k=5)
    alt_p_list = [df['data'][topic_index]['paragraphs'][i]['context'] for i in alt_p_index]
    alt_article_list = [nlp(alt_p) for alt_p in alt_p_list]
    alt_paragraph_list = [textacy.Doc(alt_p, lang='en_core_web_sm') for alt_p in alt_p_list]
        
#     # Preprocessing for different topic distractor generation
#     # choose random topic
#     topic_list = list(range(len(df['data'])))
#     # make sure isn't the same topic as current topic
#     topic_list.remove(topic_index)
#     alt_topic_index = random.choice(topic_list)
#     alt_topic = df['data'][alt_topic_index]['title']
#     alt_topic_paragraph = df['data'][alt_topic_index]['paragraphs'][0]['context']
#     # select first paragraph from topic - option to randomize selection as well
#     alt_topic_article = nlp(alt_topic_paragraph)
#     alt_topic = textacy.Doc(alt_topic_paragraph, lang='en_core_web_sm')
    
    ent_list = [str(i).lower() for i in list(article.ents)]
#     print('ent_list: ' + str(ent_list))

    if answer in ent_list:
        print('in entity list')
        ent_labels = [x.label_ for x in article.ents]
        # get all named entities in sentence
        sent_ent_list = [str(i).lower() for i in list(sentence.ents)]
        sent_labels = [x.label_ for x in sentence.ents]
#         alt_topic_ent_list = [str(i).lower() for i in list(alt_topic_article.ents)]
#         alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
        alt_article_ent_list = []
        alt_article_labels = []
        for p in alt_article_list:
            p_ent_list = [str(i).lower() for i in list(p.ents)]
            p_labels = [x.label_ for x in p.ents]
            alt_article_ent_list.extend(p_ent_list)
            alt_article_labels.extend(p_labels)
        print('alt_article_ent: ' + str(alt_article_ent_list))
        print('alt_article_labels: ' + str(alt_article_labels))
        merged=set(ent_labels+sent_ent_list+alt_article_ent_list)
#         merged=set(ent_labels+sent_ent_list+alt_article_ent_list+alt_article_ent_list)
        max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list))
        # create table of named entities
#         max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
#         print(max_length)
        ne_pd = pd.DataFrame()
        ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
        ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
        ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
        ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
        ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
        ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
#         ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
#         ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

        ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        print('answer label: ' + str(ans_label))
        alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
        alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
        alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
#         alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
        wrong_answers = set(alt_ans_list)
#         while len(wrong_answers)<3:
#             index_list.remove(alt_p_index)
#             alt_p_index = random.choice(index_list)
#             alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
#             alt_article = nlp(alt_p)
#             alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
#             alt_topic_ent_list = [str(i).lower() for i in list(alt_topic_article.ents)]
#             alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
#             alt_article_ent_list = [str(i).lower() for i in list(alt_article.ents)]
#             alt_article_labels = [x.label_ for x in alt_article.ents]
#             merged=set(ent_labels+sent_ent_list+alt_topic_ent_list+alt_article_ent_list)
#             # create table of named entities
#             max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
#     #         print(max_length)
#             ne_pd = pd.DataFrame()
#             ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
#             ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
#             ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
#             ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
#             ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
#             ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
#             ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
#             ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

#             ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
#     #         print(ans_label)
#             alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
#             alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
#             alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
#             alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
#             wrong_answers = set(alt_ans_list)
        
    else:
        print('not an entity')
        merged=[]
#         while len(merged)<3:
#             for index in alt_p_index:
#                 index_list.remove(index)
#             alt_p_index = random.choice(index_list)
#             alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
#             alt_article = nlp(alt_p)
#             alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
        correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')+'+'
        print('correct_ans_pos: ' + str(correct_ans_pos))
        doc_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
        print('doc_list: ' + str(doc_list))
        sent_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
        print('sent_list: ' + str(sent_list))
        alt_p_list=[]
        for p in alt_paragraph_list:
            p_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(p, correct_ans_pos)]
            alt_p_list.extend(p_list)
        print('alt_p_list: ' + str(alt_p_list))
#             alt_topic_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
#             print('alt_topic_list: ' + str(alt_topic_list))
        merged = set(doc_list+sent_list+alt_p_list)
        print('merged: ' + str(merged))
        
#         print('merged: ' + str(merged))
        if 'VP' in answer_type:
            if len(wrong_answers) >= 3:
                for i in merged:
                    item = nlp(i)
                    if (item[0].tag_ == ans_tag[0]) and (item[-1].pos_ == ans_pos[-1]):
                        wrong_answers.append(item)
            else:
                wrong_answers = merged
        if 'VERB' in answer_type:
            if len(wrong_answers) >= 3:
                for i in merged:
                    item = nlp(i)
                    if (item[0].tag_ == ans_pos_tag[0]):
                        wrong_answers.append(item)
            else:
                wrong_answers = merged
        if 'ADJ' in answer_type:
            hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
            hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
            alt_p_hyphen_sep_words=[]
            for p in alt_p_list:
                p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),p)
                alt_p_hyphen_sep_words.extend(p_hyphen_sep_words)
#             alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

            hyphen_words = hyphen_sep_words+hyphen_sep_words_sent+alt_p_hyphen_sep_words
#             print('hyphen_words: ' + str(hyphen_words))
            for w in hyphen_words:
                merged.add(w)
#             print('hyphen added merged: ' + str(merged))
            wrong_answers = merged
        if 'YEAR' in answer_type:
            for i in merged:
                if len(i)==4:
                    wrong_answers.append(i)
        if 'NUM' in answer_type:
            wrong_answers = [t2d.convert(str(i)).lstrip() for i in merged]
            if len(answer)==4:
                wrong_answers = [i for i in wrong_answers if len(i)==4]
        if 'TIME' in answer_type:
            wrong_answers = [t2d.convert(str(i)).lstrip()+' ' + str(correct_answer[-1]) for i in merged]
        if 'MONEY' in answer_type:
            currency = answer[0]
            print('currency: ' + str(currency))
            wrong_answers = [str(currency) + t2d.convert(str(i)).replace(' ','') for i in merged]
            print('wrong_answers: ' + str(wrong_answers))
        if 'DATE' in answer_type:
            if ans_length==1:
                month_list=random.sample(['January', 'Jan', 'February', 'March', 'April', \
                                       'May','June', 'July', 'August', 'September', \
                                       'October','November', \
                                          'December'].remove(answer.capitalize()),3)
        if 'ADP' in answer_type:
            wrong_answers = [i for i in wrong_answers if i[0]==correct_answer[0].orth_]
        if ('PROPN' in answer_type) | ('NOUN' in answer_type):
            ans_ent_list = [str(i).lower() for i in list(nlp(answer).ents)]
            ans_labels = [x.label_ for x in nlp(answer).ents]
            if ans_ent_list!=[]:
                for a in merged:
                    a_ent_list = [str(i).lower() for i in list(nlp(a.title()).ents)]
                    a_labels = [x.label_ for x in nlp(a.title()).ents]
                    ne_pd = pd.DataFrame()
                    ne_pd['ent'] = a_ent_list
                    ne_pd['label'] = a_labels
                    if nlp(a):
                        wrong_answers.append(list(ne_pd[(ne_pd['label'].isin(a_labels))]['ent']))
                wrong_answers=[item for sublist in wrong_answers for item in sublist]
            else:
                wrong_answers=merged
        else:
            wrong_answers=merged
    for i in wrong_answers:
        print('i: ' + str(i))
        print(type(i))
        for word in range(len(correct_answer)):
            print('word: ' + str(correct_answer[word].orth_))
            print(type(correct_answer[word].orth_))
            
    wrong_answers = [i for i in wrong_answers for word in range(len(correct_answer)) if (correct_answer[word].orth_ not in i)]
    wrong_answers=[item for item in wrong_answers if item not in ['a','an','the','that','it','who','what', '',""]]

    return (topic, paragraph, question, correct_answer, answer_type, random.sample(list(set(wrong_answers)),3))
       
                    

In [40]:
distractors = []
for i in range(len(autoq)):
    try:
        print('Sample: ' + str(i))
        print('Raw Answer: ' + str(autoq.loc[i]['answers']))
        answer_type = check_answer_type(autoq.loc[i]['answers'])
#         print('Answer type: ' + str(answer_type))
        topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(autoq.loc[i]['topics'], autoq.loc[i]['paragraphs'], \
                                  autoq.loc[i]['questions'], autoq.loc[i]['answers'], answer_type)
        distractors.append(wrong_answers)
        print('added distractors: ' + str(wrong_answers))
#         print('---------------------------------------------------------')
#         print('Question: ' + str(question))
#         print('Wrong answers: ' + str(wrong_answers))
#         print('\n')
    except:
        print("error occurred for sample#: " + str(i))
        distractors.append(['NA'])
autoq['distractors']=distractors
autoq['distractor_length'] = [1 if (len(autoq.loc[i]['distractors'])==1) else 0 for i in list(autoq.index.values) ]


Sample: 0
Raw Answer: 
error occurred for sample#: 0
Sample: 1
Raw Answer: 
error occurred for sample#: 1
Sample: 2
Raw Answer: 
error occurred for sample#: 2
Sample: 3
Raw Answer: 
error occurred for sample#: 3
Sample: 4
Raw Answer: 
error occurred for sample#: 4
Sample: 5
Raw Answer: the apostles
not an entity
correct_ans_pos: <DET>+<NOUN>+
doc_list: ['the works', 'a ruler', 'a nation', 'the founder', 'the apostles', 'some manifestations', 'the imamate', 'a right', 'each individual', 'some historians', 'the term', 'a term', 'the time', 'the term', 'the turn']
sent_list: ['the apostles']
alt_p_list: ['the concept', 'the possibility', 'the acceptance', 'an anarchist women', 'the organisation', 'the idea', 'a variety', 'a time', 'a surge', 'the 1960s', 'the 1960s', 'the time', 'the anarchists', 'the conflict', 'the anarchists', 'a grouping', 'an anarchist society', 'the region', 'a number', 'a manifestation', 'the struggle', 'the anarchist struggle', 'the state', 'the philosophy', 'all 

not an entity
correct_ans_pos: <NOUN>+
doc_list: ['self', 'anarchist', 'label', 'work', 'what', 'property', 'reason', 'claim', 'founder', 'anarchist theory', 'theory', 'order', 'society', 'organisation', 'coordinator', 'idea', 'order', 'wills', 'individuals', 'interests', 'quote', 'matter', 'mother', 'daughter', 'order', 'what', 'property', 'answers', 'accusation', 'property', 'theft', 'work', 'institution', 'decreed', 'property', 'propriété', 'owners', 'rights', 'abuse', 'property', 'what', 'possession', 'ownership', 'resources', 'goods', 'use', 'property', 'bulwark', 'state power', 'opposition', 'state', 'religion', 'practices', 'anarchists', 'thinkers', 'time']
sent_list: ['property', 'bulwark', 'state power']
alt_p_list: ['file', 'sketch', 'anarchist', 'form', 'anarchism', 'egoism', 'egoist anarchism', 'proponents', 'anarchism', 'text', 'philosophy', 'limitation', 'rights', 'individuals', 'power', 'what', 'regard', 'state', 'morality', 'rights', 'reification', 'fallacy)|spooks', 'm

not an entity
correct_ans_pos: <ADJ>+<NOUN>*<NOUN>+
doc_list: ['special conference', 'technical excuse', 'swiss supporters', 'tiny minority', 'his followers', 'final split', 'its headquarters', 'revolutionary anarchist programme']
sent_list: ['revolutionary anarchist programme']
alt_p_list: ['social change', 'christian anarchism', 'christian movement|movement', 'political theology', 'its main proponents', 'insurrectionary anarchism', 'radical individualism', 'best social system', 'individual freedom', 'libertarian socialism', 'political culture', 'political ideologies', 'social theories', 'first person', 'her labour', 'her needs', 'their nature', 'hegelian philosopher', 'influential early text', 'individualist anarchism', 'french anarchists', 'harsh reaction', 'social upheaval', 'nationalist uprisings', 'systematic change', 'conservative elements', 'further revolt', 'principal translator', 'later briefly', 'spanish anarchism', 'proudhonian ideas', 'early 1860s', 'spanish revolution', '

error occurred for sample#: 20
Sample: 21
Raw Answer: 
error occurred for sample#: 21
Sample: 22
Raw Answer: 
error occurred for sample#: 22
Sample: 23
Raw Answer: 
error occurred for sample#: 23
Sample: 24
Raw Answer: 
error occurred for sample#: 24
Sample: 25
Raw Answer: 
error occurred for sample#: 25
Sample: 26
Raw Answer: contemporary global International of Anarchist Federations
not an entity
correct_ans_pos: <ADJ>+<ADJ>*<PROPN>*<ADP>*<PROPN>*<PROPN>+
doc_list: ['contemporary global international of anarchist federations']
sent_list: ['contemporary global international of anarchist federations']
alt_p_list: []
merged: {'contemporary global international of anarchist federations'}
i: contemporary global international of anarchist federations
<class 'str'>
word: contemporary
<class 'str'>
word: global
<class 'str'>
word: international
<class 'str'>
word: of
<class 'str'>
word: anarchist
<class 'str'>
word: federations
<class 'str'>
error occurred for sample#: 26
Sample: 27
Raw Answ

not an entity
correct_ans_pos: <PROPN>+<NUM>+
doc_list: ['september 1945', 'may 1948', 'february 1946']
sent_list: ['september 1945']
alt_p_list: []
merged: {'september 1945', 'may 1948', 'february 1946'}
i: september 1945
<class 'str'>
word: september
<class 'str'>
word: 1945
<class 'str'>
i: may 1948
<class 'str'>
word: september
<class 'str'>
word: 1945
<class 'str'>
i: february 1946
<class 'str'>
word: september
<class 'str'>
word: 1945
<class 'str'>
error occurred for sample#: 32
Sample: 33
Raw Answer: 1950
in entity list
alt_article_ent: ['bohemian', 'nudist]]s', 'european', 'france', 'han', 'miguel gimenez', 'lev chernyi', 'marxist', 'marxist', 'german', 'the german revolution', '1918–19', 'the bavarian soviet republic', 'gustav landauer', 'silvio gesell', 'erich mühsam', 'italian', 'unione sindacale italiana', '800,000', 'the italian anarchist union', '20,000', 'umanita nova', 'daily', '\xa0', 'anarchists', 'first', 'the mexican revolution', 'the mexican liberal party', 'the ea

...

not an entity
correct_ans_pos: <PROPN>+
doc_list: ['andorra', 'andorra']
sent_list: ['andorra']
alt_p_list: ['andorra', 'france', 'spain', 'sometent', 'andorran', 'andorrans', 'andorra', 'united nations', 'un', 'organization', 'security', 'co', 'europe', 'osce', 'european union', 'eu', 'roc', 'bruixes', 'witches', 'andorra', 'canillo', 'andorra', 'andorran', 'som', 'andorra telecom', 'sta', 'andorra', 'ftth', 'catalan', 'catalan', 'andorra', 'andorran', 'som', 'andorra telecom', 'sta', 'andorra', 'ftth']
merged: {'andorran', 'osce', 'andorra', 'europe', 'catalan', 'roc', 'canillo', 'co', 'security', 'ftth', 'united nations', 'eu', 'spain', 'france', 'andorra telecom', 'andorrans', 'organization', 'sometent', 'un', 'european union', 'witches', 'sta', 'bruixes', 'som'}
i: andorran
<class 'str'>
word: andorra
<class 'str'>
i: andorra
<class 'str'>
word: andorra
<class 'str'>
i: roc
<class 'str'>
word: andorra
<class 'str'>
i: ftth
<class 'str'>
word: andorra
<class 'str'>
i: united nation

i: small states
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: marc almugaver
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: winter olympic games
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: olympic games
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: andorran army
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: french
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
i: jules baulard
<class 'str'>
word: olympic
<class 'str'>
word: games
<class 'str'>
added distractors: ['marc almugaver', 'small states', 'french']
Sample: 1847
Raw Answer: 
error occurred for sample#: 1847
Sample: 1848
Raw Answer: 
error occurred for sample#: 1848
Sample: 1849
Raw Answer: 
error occurred for sample#: 1849
Sample: 1850
Raw Answer: 
error occurred for sample#: 1850
Sample: 1851
Raw Answer: 
error occurred for sample#: 1851
Sample: 1852
Raw Answer: mean or average
not a

not an entity
correct_ans_pos: <DET>+<PROPN>*<PROPN>*<ADV>*<VERB>*<DET>*<PROPN>*<PROPN>+
doc_list: ['the afc', 'the afc championship game', 'the lamar hunt trophy', 'the afc champion then plays the nfc champion', 'the super bowl']
sent_list: ['the afc champion then plays the nfc champion', 'the super bowl']
alt_p_list: ['the oakland raiders', 'the san diego chargers', 'the afc', 'the afc', 'an updated afc', 'the afc', 'the nfc', 'the american football conference', 'the national football league', 'the united states', 'the national football conference', 'the afc', 'the nfc', 'the new england patriots', 'the kansas city chiefs', 'the houston oilers', 'the tennessee oilers', 'the tennessee titans', 'the nfc', 'the afc']
merged: {'the kansas city chiefs', 'the houston oilers', 'the super bowl', 'the national football league', 'the national football conference', 'the nfc', 'the new england patriots', 'the tennessee titans', 'the afc championship game', 'the afc champion then plays the nfc ch

not an entity
correct_ans_pos: <PROPN>+<NOUN>+
doc_list: []
sent_list: []
alt_p_list: []
merged: set()
error occurred for sample#: 1885
Sample: 1886
Raw Answer: plentiful
not an entity
correct_ans_pos: <ADJ>+
doc_list: ['young', 'plentiful', 'special', 'their personal']
sent_list: ['plentiful']
alt_p_list: ['its wartime', 'british', 'high', 'british', 'american', 'own', 'which', 'its', 'great commercial', 'international', 'significant', 'his', 'his old', 'convinced', 'better', 'orwell', 'political', 'russian', 'violent conspiratorial', 'hungry', 'radical', 'alert', 'ukrainian', 'past', 'soviet', 'essential', 'socialist', 'my', 'soviet', 'that', 'which', 'other', 'allegorical', 'stalinist', 'democratic', 'hostile', 'that', 'his', 'spanish', 'brutal', 'satirical', 'his', 'first', 'which', 'full', 'political', 'artistic']
merged: {'radical', 'political', 'their personal', 'socialist', 'satirical', 'stalinist', 'its', 'its wartime', 'his', 'my', 'special', 'artistic', 'hungry', 'soviet', '

not an entity
correct_ans_pos: <PROPN>+<PROPN>+
doc_list: ['farmer jones']
sent_list: ['farmer jones']
alt_p_list: ['second world war', 't. s. eliot', 'andré deutsch', 'animal farm', 'london letter', 'partisan review', 'manor farm', 'animal farm', 'animal farm']
merged: {'andré deutsch', 'second world war', 'partisan review', 't. s. eliot', 'animal farm', 'manor farm', 'london letter', 'farmer jones'}
i: andré deutsch
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: second world war
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: partisan review
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: t. s. eliot
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: animal farm
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: manor farm
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: london letter
<class 'str'>
word: farmer
<class 'str'>
word: jones
<class 'str'>
i: farmer

In [37]:
topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(autoq.loc[27]['topics'], autoq.loc[27]['paragraphs'], \
                                  autoq.loc[27]['questions'], autoq.loc[27]['answers'], check_answer_type(autoq.loc[27]['answers']))

not an entity
correct_ans_pos: <PROPN>+<PROPN>+
doc_list: ['benito mussolini', 'arditi del popolo', 'luigi fabbri']
sent_list: ['luigi fabbri']
alt_p_list: ['latin america', 'oaxaca uprising', 'world trade organization', 'world economic forum', 'simon critchley', 'anarchist federations', 'international workers', 'confederación general del trabajo', 'united kingdom', 'sex pistols', 'western europe', 'freetown christiania', 'david graeber', 'andrej grubacic', 'michel foucault', 'new york city', 'greenwich village', 'edna st. vincent millay', 'margaret c. anderson|margaret anderson', 'emma goldman', 'magnus hirschfeld', 'robert reitzel', 'poor devil', 'virginia bolten', 'william batchelder greene']
merged: {'western europe', 'magnus hirschfeld', 'robert reitzel', 'david graeber', 'william batchelder greene', 'latin america', 'united kingdom', 'confederación general del trabajo', 'freetown christiania', 'michel foucault', 'luigi fabbri', 'emma goldman', 'virginia bolten', 'edna st. vincent

In [41]:
wikipedia_data = {"data": [], "version" : 1.0}
print(wikipedia_data)
for topic in autoq.topics.unique():
    print('topic: ' + str(topic))
    unique_paragraphs = autoq[autoq['topics']==topic].paragraphs.unique()
    print('number of paragraphs: ' + str(len(unique_paragraphs)))
    agg = autoq[autoq['topics']==topic].groupby('paragraphs').agg({'questions': lambda x: '\n\n'.join(x), \
                                                                   'answers': lambda x: '\n'.join(x), \
                                                                   'id':lambda x: '\n'.join(x), \
                                                                  'distractors': lambda a: list(a)})
    context = []
    for item in range(len(unique_paragraphs)):
        para = unique_paragraphs[item]
        qas = []
        questions = agg.iloc[item]['questions'].split('\n\n')
        print('question')
        print(agg.iloc[item]['questions'])
        print('distractors')
        print(agg.iloc[item]['distractors'])
        for q in range(len(questions)):
            q_dict = {'question': '', 'id': '', 'answers': [], 'distractors': []}
            q_dict['question'] = questions[q]
            q_dict['id'] = agg.iloc[item]['id'].split('\n')[q]
            ans_dict = {'answer_start': 0, 'text': ''}
            ans_dict['text']= agg.iloc[item]['answers'].split('\n')[q]
            q_dict['answers'].append(ans_dict)
            qas.append(q_dict)
            q_dict['distractors'] = agg.iloc[item]['distractors'][q]
        context_dict = {'context': para, 'qas' : qas}
        context.append(context_dict)
    wikipedia_data['data'].append({'title' : topic, 'paragraphs' : context})

with open('wiki_squad_00-mc.json', 'w') as outfile:  
    json.dump(wikipedia_data, outfile)

{'data': [], 'version': 1.0}
topic: Anarchism
number of paragraphs: 84
question





distractors
[['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
in what year was the international of conference held ?
distractors
[['1793', '1960s', '1970s']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
when was the uruguayan anarchist federation founded ?

when was the mexican anarchist federation established ?

when was the league of free social constructors formed ?

when was the dissolved federation formed ?
distractors
[['february 1946', 'the 1960s', '1947'], ['the 19th century', '1947', 'the early 1940s'], ['NA'], ['february 1946', '1947', '1945']]
question
what type of programme does imier congress adopt ?
distractors
[['radical individualism', 'insurrectionary anarchism', 'tiny minority']]
question

distractors
[['NA']]
question

d

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
topic: Alabama
number of paragraphs: 144
question







distractors
[['NA'], ['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the highest poverty rate in the us ?
distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the name of the region that the northwestern lies of ?
distractors
[['republicans', 'terri sewell', 'tuscaloosa']]
question
what was the population of michigan in 2010 ?
distractors
[['one', '100,000', '87,451']]
question

distractors
[['NA']]
question
what university has the university of michigan ?
distractors
[['large', 'lower', 'partisan']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
ques


distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what was the nickname given to lincoln ?

when was robert todd lincoln born ?

who was the last ancestor of the lincolns' last ancestor ?

when was thomas `` tad '' born ?
distractors
[['robert', 'robert lincoln', 'robert todd lincoln'], ['december 21, 1850', '1864 spring', 'august 16, 1837'], ['edward baker lincoln', 'horace greeley', 'new york tribune'], ['december 21, 1850', 'february 1, 1850', 'may 1856']]
question

distractors
[['NA']]
question
who james james j james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james ?
distractors
[['james g. randall', 'civil war', 'california']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

di

question

distractors
[['NA']]
question
plato argued that there are not a part of what ?

what does the form of apple exist within each apple ?

plato argued that all things have a universal form to what ?

where can we look a form of an apple ?

is there a proper form of a universal form ?

what is a particular form of an apple and a universal form of an apple ?

what did plato call the world of forms ?
distractors
[['specialised knowledge', 'small selection', 'universal form'], ['politics', 'lesbos', 'athens'], ['the common writing medium', 'a book', 'the groundwork'], ['of impetus', 'in this distinction', 'of riddles'], ['low', 'convergent', 'lowest'], ['possible', 'next', 'central'], ['a thing', 'an eye', 'the world']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distracto

question
who has argued that darwinism is one possible form of love ?
distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what term was coined by the book `` pathological altruism '' ?
distractors
[['brahma sutras,', 'swami sivananda,', 'swami vivekananda,']]
question
who coined `` altruism '' ?
distractors


question

distractors
[['NA']]
topic: Anthropology
number of paragraphs: 81
question



distractors
[['NA'], ['NA']]
question
the patterns of social relationships in one or more human cultures can refer to what ?
distractors
[['lineages", "', 'anthropology.', 'anthropologists,']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the strategy in afghanistan ?
distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what discipline is anthropology ?

what is the scientific status of the human community ?
distractors
[['most', 'verbal', 'daily'], ['psychological harm', 'other hand', 'cultural relativism']]
question
what is the study of humans and humans in the past and present ?

what does linguistic anthropology affect social life ?
distractors
[['1960s', 'material', 'classic'], ['fields', 'others', 'monographs']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distracto

number of paragraphs: 45
question





distractors
[['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is produced to win or win the award for the award ?
distractors
[['pacific data images', 'walt disney productions', 'united kingdom']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
where did spielberg film film come from ?
distractors
[['NA']]
question

distractors
[['NA']]
question

distractors

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the name of the person who has a relationship with the 360 ?
distractors
[['lleyton hewitt', 'andre agassi charitable foundation', 'canadian']]
question

distractors
[['NA']]
topic: Austroasiatic languages
number of paragraphs: 39
question





distractors
[['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
in what year did the genetic and linguistic research in east asia suggest an origin and linguistic research ?
distractors
[['2018', '2013', '2011']]
question

distractors
[['NA']]
question
what is the range of migration from southeast asia ?
distractors
[['10,000 years', '0.6 kya', '4.3\u2009±\u20090.2 kya']]
question
what was the name of the region in southern indochina by the end of the 


distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
when was the first `` animal farm '' published ?
distractors
[['1985', 'august 2017', '1917 to 1943']]
question

distractors
[['NA']]
question
what food is the farm running of food ?
distractors
[['young', 'violent conspiratorial', 'their personal']]
question
how long later did some men attack animal farm ?
distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what was the name of the farm that the farm was named ?
distractors
[['NA']]


### Old Function

In [16]:
answer =autoq.loc[110]['answers']
article = nlp(unidecode.unidecode(autoq.loc[110]['paragraphs']))

topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == 'Autism'), None)
print(topic_index)
index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
print(len(index_list))
# make sure isn't the same paragraph as current paragraph
p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                if d["context"] == autoq.loc[110]['paragraphs']), None)
index_list.remove(p_index)
print('p_index removed: ' + str(len(index_list)))
# Choose 5 random paragraphs from the same article
alt_p_index = random.choices(index_list,k=5)
print('random indices: ' + str(alt_p_index))
alt_p_list = [df['data'][topic_index]['paragraphs'][i]['context'] for i in alt_p_index]
print(alt_p_list)
alt_article_list = [nlp(alt_p) for alt_p in alt_p_list]

ent_list = [str(i).lower() for i in list(article.ents)]
print('ent_list: ' + str(ent_list))

if answer in ent_list:
    print('in entity list')
    ent_labels = [x.label_ for x in article.ents]
    alt_article_ent_list = []
    alt_article_labels = []
    for p in alt_article_list:
        p_ent_list = [str(i).lower() for i in list(p.ents)]
        p_labels = [x.label_ for x in p.ents]
        alt_article_ent_list.extend(p_ent_list)
        alt_article_labels.extend(p_labels)
    print('alt_article_ent: ' + str(alt_article_ent_list))
    print('alt_article_labels: ' + str(alt_article_labels))
    merged=set(ent_labels+alt_article_ent_list)
#         merged=set(ent_labels+sent_ent_list+alt_article_ent_list+alt_article_ent_list)
    max_length = max(len(ent_list), len(alt_article_ent_list))
    # create table of named entities
#         max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
#         print(max_length)
    ne_pd = pd.DataFrame()
    ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
    ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
    ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
    ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))


    ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
    print('answer label: ' + str(ans_label))
    alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
    alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
#         alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
print(alt_ans_list)

1
70
p_index removed: 69
random indices: [15, 62, 39, 63, 59]
['Several lines of evidence point to synaptic dysfunction as a cause of autism. Some rare mutations may lead to autism by disrupting some synaptic pathways, such as those involved with cell adhesion. Gene replacement studies in mice suggest that autistic symptoms are closely related to later developmental steps that depend on activity in synapses and on activity-dependent changes. All known teratogens (agents that cause birth defects) related to the risk of autism appear to act during the first eight weeks from conception, and though this does not exclude the possibility that autism can be initiated or affected later, there is strong evidence that autism arises very early in development.', 'The New Latin word "autismus" (English translation "autism") was coined by the Swiss psychiatrist Eugen Bleuler in 1910 as he was defining symptoms of schizophrenia. He derived it from the Greek word "autós" (αὐτός, meaning "self"), and u

In [13]:
topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == 'Autism'), None)
print(topic_index)
index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
print(len(index_list))
# make sure isn't the same paragraph as current paragraph
p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                if d["context"] == autoq.loc[110]['paragraphs']), None)
index_list.remove(p_index)
print('p_index removed: ' + str(len(index_list)))
# Choose 5 random paragraphs from the same article
alt_p_index = random.choices(index_list,k=5)
print('random indices: ' + str(alt_p_index))
alt_p_list = [df['data'][topic_index]['paragraphs'][i]['context'] for i in alt_p_index]
print(alt_p_list)
alt_paragraph_list = [textacy.Doc(alt_p, lang='en_core_web_sm') for alt_p in alt_p_list]

answer = autoq.loc[110]['answers']
correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')+'+'
print('correct_ans_pos: ' + str(correct_ans_pos))
alt_p_list=[]
for p in alt_paragraph_list:
    p_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(p, correct_ans_pos)]
    alt_p_list.extend(p_list)
print('alt_p_list: ' + str(alt_p_list))

1
70
p_index removed: 69
random indices: [49, 14, 31, 46, 42]
["Medications may be used to treat ASD symptoms that interfere with integrating a child into home or school when behavioral treatment fails. They may also be used for associated health problems, such as ADHD or anxiety. More than half of US children diagnosed with ASD are prescribed psychoactive drugs or anticonvulsants, with the most common drug classes being antidepressants, stimulants, and antipsychotics. The atypical antipsychotic drugs risperidone and aripiprazole are FDA-approved for treating associated aggressive and self-injurious behaviors. However, their side effects must be weighed against their potential benefits, and people with autism may respond atypically. Side effects, for example, may include weight gain, tiredness, drooling, and aggression. SSRI antidepressants, such as fluoxetine and fluvoxamine, have been shown to be effective in reducing repetitive and ritualistic behaviors, while the stimulant medicati

In [134]:
for i in range(len(sample100)):
    try:
        print('Sample: ' + str(i))
        print('Raw Answer: ' + str(sample100.loc[i]['answers']))
        answer_type = check_answer_type(sample100.loc[i]['answers'])
        print('Answer type: ' + str(answer_type))
        topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(sample100.loc[i]['topics'], sample100.loc[i]['paragraphs'], \
                                  sample100.loc[i]['questions'], sample100.loc[i]['answers'], answer_type)
        print('---------------------------------------------------------')
        print('Question: ' + str(question))
        print('Wrong answers: ' + str(wrong_answers))
        print('\n')
    except:
        print("error occurred for sample#: " + str(i))

Sample: 0
Raw Answer: North Carolina and New Mexico
Answer type: ['PROPN']
---------------------------------------------------------
Question: In what states are pharmacist clinicians given prescriptive and diagnostic authority?
Wrong answers: ['indian health service', 'north carolina', 'new mexico']


Sample: 1
Raw Answer: non-specific
Answer type: ['ADJ']
---------------------------------------------------------
Question: The innate immune system responds in a generic way, meaning it is what?
Wrong answers: ['its great', 'innate immune', 'long-lasting']


Sample: 2
Raw Answer: charter
Answer type: ['NOUN']
---------------------------------------------------------
Question: Granting what status would allow private non-religious schools in the US to receive public funds?
Wrong answers: ['list', 'constitution', 'economy']


Sample: 3
Raw Answer: pigment-filled plastids responsible for the bright colors seen in flowers and ripe fruit
Answer type: ['ADJ']
---------------------------------

Answer type: ['ADJ']
---------------------------------------------------------
Question: What health problem did Tesla have in 1879?
Wrong answers: ['subsequent claims', 'same time', 'his family']


Sample: 36
Raw Answer: four
Answer type: ['NUM']
---------------------------------------------------------
Question: How many auricles do most species have?
Wrong answers: ['2', '8', 'two']


Sample: 37
Raw Answer: Supreme Court of the United States
Answer type: ['PROPN']
---------------------------------------------------------
Question: A decision made by what entity restored Tesla's patents?
Wrong answers: ['united states', 'oliver lodge', 'the united states']


Sample: 38
Raw Answer: Charles F. Peck
Answer type: ['PROPN']
---------------------------------------------------------
Question: Who else did Tesla make the acquaintance of in 1886?
Wrong answers: ['hotel new yorker', 'tesla electric company', 'petroleum exporting countries']


Sample: 39
Raw Answer: second most commonly
Answer 

error occurred for sample#: 66
Sample: 67
Raw Answer: Kenya's various ethnic groups typically speak their mother tongues within their own communities
Answer type: ['PROPN']
---------------------------------------------------------
Question: What language is spoken in Kenya?
Wrong answers: ['kenya', 'their mother tongues', 'armed forces']


Sample: 68
Raw Answer: flattened circular
Answer type: ['VP']
---------------------------------------------------------
Question: What shape are granal thylakoids?
Wrong answers: ['are continuous', 'retained fundamental', 'was due']


Sample: 69
Raw Answer: artifact
Answer type: ['ADJ']
---------------------------------------------------------
Question: What is the force called rgarding a potential field between two locations?
Wrong answers: ['british', 'earliest', 'civil']


Sample: 70
Raw Answer: force
Answer type: ['NOUN']
---------------------------------------------------------
Question: What notion keeps it's meaning through both Netonian and S

---------------------------------------------------------
Question: Los Angeles is in the lower part of what?
Wrong answers: ['serbian cyrillic', 'southern california', 'никола']


Sample: 99
Raw Answer: Veni redemptor gentium
Answer type: ['NOUN']
---------------------------------------------------------
Question: What was the hymn based on?
Wrong answers: ['oil shock', 'term effects', 'redemptor gentium']




In [27]:
index_list = list(range(len(df['data'][0]['paragraphs'])))
random.choices(index_list, k=5)

[72, 65, 21, 42, 38]

In [ ]:
# For Debugging
topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(autoq.loc[32]['topics'], autoq.loc[32]['paragraphs'], \
                                  autoq.loc[32]['questions'], autoq.loc[32]['answers'], check_answer_type(autoq.loc[32]['answers']))
print(topic, paragraph, question, correct_answer, answer_type, wrong_answers)

In [70]:
def long_generate_distractor(topic, paragraph, question, answer, answer_type):
    wrong_answers=[]
    #preprocess answer
    correct_answer = nlp(preprocess(answer))
    # get answer pos
    ans_length = len(correct_answer)
    ans_tag = [token.tag_ for token in correct_answer]
    ans_pos = [token.pos_ for token in correct_answer]
    # tokenize paragraph
    article = nlp(unidecode.unidecode(paragraph))
    doc = textacy.Doc(paragraph, lang='en_core_web_sm')
        
    # Preprocessing for same sentence distractor generation
    # get all sentences in paragraph
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_list = sent_detector.tokenize(paragraph.strip())

    # find sentence that has answer
    for s in sent_list:
        if answer in s:
            ans_sent = s
    # tokenize sentence
    sentence = nlp(ans_sent)
    sent = textacy.Doc(sentence, lang='en_core_web_sm')
        
    # Preprocessing for same topic distractor generation
    # choose random paragraph from same topic
    topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == topic), None)
    index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
    # make sure isn't the same paragraph as current paragraph
    p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                    if d["context"] == paragraph), None)
    index_list.remove(p_index)
    alt_p_index = random.choice(index_list)
    alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
    alt_article = nlp(alt_p)
    alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
        
    # Preprocessing for different topic distractor generation
    # choose random topic
    topic_list = list(range(len(df['data'])))
    # make sure isn't the same topic as current topic
    topic_list.remove(topic_index)
    alt_topic_index = random.choice(topic_list)
    alt_topic = df['data'][alt_topic_index]['title']
    alt_topic_paragraph = df['data'][alt_topic_index]['paragraphs'][0]['context']
    # select first paragraph from topic - option to randomize selection as well
    alt_topic_article = nlp(alt_topic_paragraph)
    alt_topic = textacy.Doc(alt_topic_paragraph, lang='en_core_web_sm')
    
    if 'VP' in answer_type:
        vb_pattern = r'<VERB>+<ADJ>*<ADV>*<PART>*<DET>*<NOUN>+<ADP>*<DET>*<NOUN>*'
        vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, vb_pattern)]
#         print('vb_list: ' + str(vb_list))
        sent_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, vb_pattern)]
#         print('sent_vb_list: ' + str(sent_vb_list))
        alt_paragraph_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, vb_pattern)]
#         print('alt_paragraph_vb_list: ' + str(alt_paragraph_vb_list))
        alt_topic_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, vb_pattern)]  
#         print('alt_topic_vb_list: ' + str(alt_topic_vb_list))
        merged = set(vb_list+sent_vb_list+alt_paragraph_vb_list+alt_topic_vb_list)
        for i in merged:
#             print('i: ' + str(i))
            item = nlp(i)
#             print('item[0].tag_ '+str(item[0].tag_))
#             print('ans_tag[0] '+str(ans_tag[0]))
#             print('item[-1].pos_ '+str(item[-1].pos_))
#             print('ans_pos[-1] '+str(ans_pos[-1]))
            if (item[0].tag_ == ans_tag[0]) and (item[-1].pos_ == ans_pos[-1]):
                wrong_answers.append(item)
        if len(wrong_answers) < 3:
                wrong_answers = merged
    elif 'VERB' in answer_type:
        verbs = [token.orth_.lower() for token in article if (token.pos_=='VERB')]
        sent_verbs = verbs = [token.orth_.lower() for token in sentence if (token.pos_=='VERB')]
        alt_paragraph_verbs = [token.orth_.lower() for token in alt_article if (token.pos_=='VERB')]
        alt_topic_verbs = [token.orth_.lower() for token in alt_topic_article if (token.pos_=='VERB')]
        merged = set(verbs+sent_verbs+alt_paragraph_verbs+alt_topic_verbs)
        for i in merged:
            item = nlp(i)
            if (item[0].tag_ == ans_pos_tag[0]):
                wrong_answers.append(item)
    elif 'ADJ' in answer_type:
        if ans_length==1:
            single_adj = [token.orth_.lower() for token in article if token.pos_=='ADJ']
            single_adj_sent = [token.orth_.lower() for token in sentence if token.pos_=='ADJ']
            alt_paragraph_single_adj = [token.orth_.lower() for token in alt_article if token.pos_=='ADJ']
            alt_topic_single_adj = [token.orth_.lower() for token in alt_topic_article if token.pos_=='ADJ']
            wrong_answers = set(single_adj+single_adj_sent+alt_paragraph_single_adj+alt_topic_single_adj)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
            #adj_pattern = r'<ADV>*<ADJ>+<PART>*<DET>*<NOUN>*'
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]

                hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
                hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
                alt_p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_p)
                alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

                adj_list.extend(hyphen_sep_words)
                sent_adj_list.extend(hyphen_sep_words_sent)
                alt_paragraph_adj_list.extend(alt_p_hyphen_sep_words)
                alt_topic_adj_list.extend(alt_topic_hyphen_sep_words)
                wrong_answers = set(adj_list+sent_adj_list+alt_paragraph_adj_list+alt_topic_adj_list)
    elif 'ADV' in answer_type:
        if ans_length==1:
            single_adj = [token.orth_.lower() for token in article if token.pos_=='ADV']
            single_adj_sent = [token.orth_.lower() for token in sentence if token.pos_=='ADV']
            alt_paragraph_single_adj = [token.orth_.lower() for token in alt_article if token.pos_=='ADV']
            alt_topic_single_adj = [token.orth_.lower() for token in alt_topic_article if token.pos_=='ADV']
            wrong_answers = set(single_adj+single_adj_sent+alt_paragraph_single_adj+alt_topic_single_adj)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
            #adj_pattern = r'<ADV>*<ADJ>+<PART>*<DET>*<NOUN>*'
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]

                hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
                hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
                alt_p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_p)
                alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

                adj_list.extend(hyphen_sep_words)
                sent_adj_list.extend(hyphen_sep_words_sent)
                alt_paragraph_adj_list.extend(alt_p_hyphen_sep_words)
                alt_topic_adj_list.extend(alt_topic_hyphen_sep_words)
                wrong_answers = set(adj_list+sent_adj_list+alt_paragraph_adj_list+alt_topic_adj_list)
    elif 'NOUN' in answer_type:
        if ans_length==1:
            single_noun = [token.orth_.lower() for token in article if (token.pos_=='NOUN')]
            single_noun_sent = [token.orth_.lower() for token in sentence if (token.pos_=='NOUN')]
            alt_single_noun = [token.orth_.lower() for token in alt_article if (token.pos_=='NOUN')]
            alt_topic_single_noun = [token.orth_.lower() for token in alt_topic_article if (token.pos_=='NOUN')]
            wrong_answers = set(single_noun+single_noun_sent+alt_single_noun+alt_topic_single_noun)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
                wrong_answers = set(np_list+sent_np_list+alt_paragraph_np_list+alt_topic_np_list)
    elif 'YEAR' in answer_type:
        number_list = [token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        for i in merged:
            if len(i)==4:
                wrong_answers.append(i)
    elif 'NUM' in answer_type:
        number_list = [token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        wrong_answers = [t2d.convert(str(i)).lstrip() for i in merged]
    elif 'TIME' in answer_type:
        time_list = [token.orth_ for token in article if token.pos_=='NUM']
        time_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_time_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_time_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(time_list+time_list_sent+alt_paragraph_time_list+alt_topic_time_list)
        print('merged: ' + str(merged))
        wrong_answers = [t2d.convert(str(i)).lstrip()+' ' + str(correct_answer[-1]) for i in merged]
    elif 'MONEY' in answer_type:
        currency = answer[0]
        number_list = [str(currency) + token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [str(currency) + token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [str(currency) + token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [str(currency) + token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        wrong_answers = [t2d.convert(str(i)).replace(' ','') for i in merged]
    elif 'DATE' in answer_type:
        if ans_length==1:
            month_list=random.sample(['January', 'Jan', 'February', 'March', 'April', \
                                   'May','June', 'July', 'August', 'September', \
                                   'October','November', \
                                      'December'].remove(answer.capitalize()),3)
        else:
            date_regex = r'<NUM>*<PROPN>+<NUM>*<PUNCT>*<NUM>+'
            date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, date_regex)]
            sent_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, date_regex)]
            alt_paragraph_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, date_regex)]
            alt_topic_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, date_regex)]
            wrong_answers = set(date_list+sent_date_list+alt_paragraph_date_list+alt_topic_date_list)
    elif 'PROPN' in answer_type:
        ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
        if answer in ent_list:
            ent_labels = [x.label_ for x in article.ents]
            # get all named entities in sentence
            sent_ent_list = [str(i) for i in list(sentence.ents)]
            sent_labels = [x.label_ for x in sentence.ents]
            alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
            alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
            alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
            alt_article_labels = [x.label_ for x in alt_article.ents]

            # create table of named entities
            max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
    #         print(max_length)
            ne_pd = pd.DataFrame()
            ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
            ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
            ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
            ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

            ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
    #         print(ans_label)
            alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
            alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
            wrong_answers = set(alt_ans_list) - set([str(correct_answer)])
        else:
            correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
            np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
            sent_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
            alt_paragraph_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
            alt_topic_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
            wrong_answers = set(np_list+sent_np_list+alt_paragraph_np_list+alt_topic_np_list)
    else:
        correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
        propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
        sent_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
        alt_paragraph_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
        alt_topic_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
        wrong_answers = set(propn_list+sent_propn_list+alt_paragraph_propn_list+alt_topic_propn_list)
        if 'ADP' in answer_type:
            wrong_answers = [i for i in wrong_answers if i[0]==correct_answer[0].orth_]
    wrong_answers = [i for i in wrong_answers for word in range(len(correct_answer)) if correct_answer[word].orth_ not in i]
                
    return (topic, paragraph, question, correct_answer, answer_type, set(wrong_answers))
        